# Packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from matplotlib import cm
import pandas as pd

from os import listdir
from os.path import isfile, join
#from pycaret.anomaly import *
import plotly.graph_objects as go
import plotly.express as px
import time

# Input Average Speed Torque

In [ ]:
speed = scipy.io.loadmat("/home/parker/data/HUMS2023/Input_Average_Speed_Torque.mat")
speed

## torque load

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
x = range(len(speed["TT"][0]))
y = speed["TT"][0]
plt.title("torque load")
plt.xlabel("file number")
plt.plot(x,y)
plt.show()

## shaft speed

In [ ]:
x = range(len(speed["Shaft_Speed"][0]))
y = speed["Shaft_Speed"][0]
plt.xlabel("shaft speed")
plt.title("torque load")
plt.plot(x,y)
plt.show()

# Gear Box Oil Temp

In [ ]:
temp = scipy.io.loadmat("/home/parker/data/HUMS2023/05_Gearbox_oil_temp.mat")

In [ ]:
temp

In [ ]:
x = range(len(temp["TC_in"][0]))
y = temp["TC_in"][0]
plt.title("Gearbox inlet oil temperature in Deg C")
plt.xlabel("Cycle")
plt.plot(x,y)
plt.show()

In [ ]:
x = range(len(temp["TC_out"][0]))
y = temp["TC_out"][0]
plt.title("Gearbox outlet oil temperature in Deg C")
plt.xlabel("Cycle")
plt.plot(x,y)
plt.show()

# Data Processing <a class="anchor" id="dataprocessing"></a>

In [ ]:
def create_data_frame(data_path):
    """
    This function aims to create a dataframe
    
    INPUT:
         + data_path: the path of the data folder
    OUTPUT:
         + a dataframe that contains 5 columns: File, Day, Date, Time, Timestamp
    """
    file_names = [f for f in listdir(data_path) if isfile(join(data_path, f))] #Get all filenames in the folder
    image_names = [f for f in file_names if f[-3:] == "mat"] #Get all image names by filering out unnecessary files
    
    df = pd.DataFrame() #Create the dataframe
    file_paths = []     #Store image path to read image
    days = []           #Store days (e.g DAY01, Day 02)
    dates = []          #Store date (e.g 2021-12-08)
    hours = []          #Store time (e.g 10:47:55)
    
    for file in image_names:
        if len(file.split("_")) == 5:
            day, a, b, date, hour = file.split("_")
            file_paths.append(data_path+file)
            days.append(day)
            dates.append(date)
            hours.append(hour[:-4])
    df["File"] = file_paths
    df["Day"] = days
    df["Date"] = dates
    df["Time"] = hours
    df["Date"] = df["Date"].apply(lambda x: x[:4]+ '-'+ x[4:6]+ '-' + x[6:])
    df["Time"] = df["Time"].apply(lambda x: x[:2]+ ':'+ x[2:4]+ ':' + x[4:])
    df["Timestamp"] = pd.to_datetime(df['Date'] + ' ' + df['Time']) #Create Timestamp to sort the dataframe by time
    df = df.sort_values(by=['Timestamp'])
    return df

In [ ]:
def data_loader(data_path, sensor):
    """
    Input_path:
        data_path: path to the data folder
        sensor: The sensor that we want to load the data (e.g. Pinion, BevelSun,Planets, Ring)
    
    
    Output: A dataframe that contains wave vales and timestamps
    """
    #Create a dataframe
    df = create_data_frame(data_path)
    
    sensors = {"IP-1": 0,
         "RF-2": 1,
         "RL-3": 2,
         "RR-4": 3}
    data = []
    timestamps = []
    for i in range(len(df)-3,len(df)):
        file_path  = df.iloc[i].File
        mat = scipy.io.loadmat(file_path)['xah'][0]
        wave = mat[sensors[sensor]].reshape(1, 405405)[0]
        data += list(wave)

    start_time = time.time()
    print("Done!")
    print("--- %s seconds ---" % (time.time() - start_time))
    return pd.DataFrame({sensor: data})

In [ ]:
data_path = "/home/parker/data/HUMS2023/"
sensor = "IP-1" #Pinion, BevelSun, Planets, Ring
data = data_loader(data_path, sensor)

# Moving Average Visualization

In [ ]:
def moving_average(data):
    data['MA64'] = data[data.columns[0]].rolling(64).mean()
    data['MA16'] = data[data.columns[0]].rolling(16).mean()
    fig = px.line(data,y=[data.columns[0], "MA16", "MA64"], title=data.columns[0], template = 'plotly_dark',
                  width=900, height=600)
    fig.write_image("./fig1.pdf")
    fig.show()
    
    print("done!")
moving_average(data)